
#Next Word Prediction using Bi-Gram and Tri-Gram model [Alice.txt data]

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk import bigrams, trigrams
from nltk import word_tokenize
from collections import Counter

    
text = '/content/alice.txt'


with open(text) as f:
    p = f.read()   
    tokens = word_tokenize(p)
    size = len(tokens)
    bi = list(bigrams(tokens, pad_left=True, pad_right=True)) # Get the padded bigrams
    size_bi = len(bi)
    
    tri = list(trigrams(tokens, pad_left=True, pad_right=True))
    size_tri = len(tri)
    
    string = str(input())
    tok = word_tokenize(string)
    
    bi_str = list(bigrams(tok, pad_left=True, pad_right=True))
    size_bi_str = len(bi_str)
    bi_test = bi_str
    
    tri_str = list(trigrams(tok, pad_left=True, pad_right=True))
    tri_test = tri_str
    size_tri_str = len(tri_str)
    
    prob_bi = (tokens.count(tok[0])/size) * (bi.count(bi_str[1])/size_bi)
    
    search = bi_str[size_bi_str-1][0]
    maxi = 0
    for sublist in bi:
        if sublist[0] == search:
            bi_test[size_bi_str-1] = sublist
            for i in range(2, len(bi_test)-1):
                prob_bi = prob_bi * (bi.count(bi_test[i])/tok.count(bi_test[i][0]))
                
            if ( maxi < prob_bi):
                maxi = prob_bi
                string_final = bi_test
    #print (string_final)
    print ("bigram: " + string + " " + string_final[len(bi_test)-1][1])
    
    search = tri_str[size_tri_str-2][0:1]
    maxi = 0
    prob_tri = prob_bi
    for sublist in tri:
        if sublist[0:1] == search:
            tri_test[size_tri_str-2] = sublist
            tri_test = tri_test[0:size_tri_str-1]
            for i in range(2, len(tri_test)-1):
                prob_tri = prob_tri * (tri.count(tri_test[i])/size_tri)
            
            if ( maxi < prob_bi):
                maxi = prob_tri
                string_final = tri_test

    print ("trigram: " + string + " " + string_final[len(tri_test)-1][2])

There is
bigram: There is it
trigram: There is nothing


# Next Word Prediction with LSTM model- [Using BSD Corpus]

## Data preparation

In [9]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM
import pickle
from tensorflow.keras.optimizers import RMSprop

In [10]:
# Import the data set, and get all sentences
import pandas as pd

df = pd.read_json('/content/train.json')
print(df['tag'].unique())

['phone call' 'meeting' 'face-to-face conversation' 'general chatting'
 'presentation' 'training']


In [11]:
df

,id,tag,title,original_language,conversation
0,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,"[{'no': 1, 'ja_speaker': '土井さん', 'en_speaker':..."
1,190329_J06_11,meeting,株主総会に関する役員向け説明（２）,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
2,190329_J06_10,meeting,株主総会に関する役員向け説明（１）,ja,"[{'no': 1, 'ja_speaker': '友井さん', 'en_speaker':..."
3,190329_J06_09,face-to-face conversation,信託銀行から株主名簿を受領,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
4,190329_J06_08,face-to-face conversation,株主総会への臨場要請,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
...,...,...,...,...,...
665,190315_E001_05,presentation,Presentation: Company introduction,en,"[{'no': 1, 'en_speaker': 'Mr. Asano Yuki', 'ja..."
666,190315_E001_04,training,Training: training a new employee,en,"[{'no': 1, 'en_speaker': 'Mr. Ben Sherman', 'j..."
667,190315_E001_03,general chatting,General chatting: Company Christmas party,en,"[{'no': 1, 'en_speaker': 'Ms. Amy Lin', 'ja_sp..."
668,190315_E001_02,phone call,Phone call: requesting a lunch meeting.,en,"[{'no': 1, 'en_speaker': 'Mr. John Smith', 'ja..."


In [12]:
df_convs = pd.concat([pd.json_normalize(df['conversation'][i]) for i in range(len(df))],ignore_index=True)
trainen = df_convs["en_sentence"].values.tolist()
trainja = df_convs["ja_sentence"].values.tolist()

In [13]:
df_convs

,no,ja_speaker,en_speaker,ja_sentence,en_sentence
0,1,土井さん,Doi-san,はい、K社システム開発部です。,Hi this is the systems development department ...
1,2,高市さん,Takaichi-san,H社の高市と申します。,My name is Takaichi from Company H.
2,3,高市さん,Takaichi-san,いつもお世話になっております。,Thank you as always.
3,4,土井さん,Doi-san,こちらこそ、お世話になっております。,Thank you as always as well.
4,5,高市さん,Takaichi-san,稲田さんはいらっしゃいますか？,Is Inada-san there?
...,...,...,...,...,...
19995,28,ボブ クックさん,Mr. Bob Cook,今日の5時までに書き上げます。,I'll finish it by 5 o'clock today.
19996,29,アル ジョンソンさん,Mr. Al Johnson,完璧だね。,Perfect.
19997,30,アル ジョンソンさん,Mr. Al Johnson,明日ゼネラルマネージャーにA社の件を話しておきますね。,I'll go talk to the general manager tomorrow t...
19998,31,アル ジョンソンさん,Mr. Al Johnson,彼がA社を気に入ってくれて、投資できる方向に決まることを願いましょう。,Hopefully he will like Company A and make the ...


In [14]:
len(trainen)

20000

In [15]:
trainen[140:160]

['I hope so.',
 'Okay, anyways, we will send 2 police officers from our side to the general meeting of shareholders on the day.',
 'Thank you very much.',
 'I will have them speak to your reception around 9:30 in the morning.',
 'Okay.',
 'I will be stationed at the reception ten minutes before the start of the general meeting so please let me know.',
 'Okay, got it.',
 'Alright, then I look forward to working with you on the day.',
 'Thank you for calling, this is the Judicial Scrivener office T.',
 'Thank you as always, my name is Takada from Company I.',
 'Thank you.',
 'Sorry but is Mr. Takanashi in today?',
 'Please wait a moment.',
 'Thank you for holding, this is Takanashi.',
 'Thank you as always, this is Takada.',
 "I'm calling today in regards to a registration request.",
 'I exercised the stock option last month and the stock increased.',
 "And I'd like to ask you to do the registration process for changing the total number of stock issued.",
 'Okay.',
 'I will send you an e

In [16]:
#concat all english messages
text = ' '.join(trainen[0:10000]) # Out of RAM if we use all the data
print('length of the corpus is: :', len(text))

length of the corpus is: : 532978


In [17]:
# tokenizer
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [18]:
# get unique words
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [19]:
len(unique_words)

6197

In [20]:
unique_words[1000:1020]

array(['Shanghai', 'She', 'Shibata', 'Shibuya', 'Shiota', 'Shiraishi',
       'Shiroki', 'Shirota', 'Shizuo', 'Shodoshima', 'Shop', 'Should',
       'Shouldn', 'Shuji', 'Silence', 'Silicone', 'Similar', 'Since',
       'Sir', 'Six'], dtype='<U16')

## Feature Engineering
- Prepare the model input (x) and output (y)
- Prepare word2id and id2word dictionaries

In [21]:
# model input = prev_words
# model output = next_words

WORD_LENGTH = 5
next_words = []
prev_words = []
for j in range(len(words) - WORD_LENGTH):
     prev_words.append(words[j:j + WORD_LENGTH])
     next_words.append(words[j + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['Hi', 'this', 'is', 'the', 'systems']
development


In [22]:
# convert features into numpy arrays
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
# prepare word2id index
for i, each_words in enumerate(prev_words):
   for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
   Y[i, unique_word_index[next_words[i]]] = 1
word2id = unique_word_index

In [23]:
# prepare id2word index ==> for inference
id2word = {y:x for x,y in unique_word_index.items()}

## Training

In [24]:
# build model
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [25]:
# train model
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=100, shuffle=True).history

Epoch 1/100
769/769 [==============================] - 16s 12ms/step - loss: 5.8374 - accuracy: 0.1256 - val_loss: 5.5766 - val_accuracy: 0.1629
Epoch 2/100
769/769 [==============================] - 9s 11ms/step - loss: 5.4303 - accuracy: 0.1797 - val_loss: 5.9845 - val_accuracy: 0.1706
Epoch 3/100
769/769 [==============================] - 9s 11ms/step - loss: 5.3909 - accuracy: 0.2084 - val_loss: 6.0799 - val_accuracy: 0.1781
Epoch 4/100
769/769 [==============================] - 9s 11ms/step - loss: 5.1424 - accuracy: 0.2396 - val_loss: 6.1607 - val_accuracy: 0.1756
Epoch 5/100
769/769 [==============================] - 9s 11ms/step - loss: 4.8766 - accuracy: 0.2762 - val_loss: 6.2780 - val_accuracy: 0.1640
Epoch 6/100
769/769 [==============================] - 9s 11ms/step - loss: 4.6221 - accuracy: 0.3148 - val_loss: 6.3553 - val_accuracy: 0.1648
Epoch 7/100
769/769 [==============================] - 9s 11ms/step - loss: 4.3895 - accuracy: 0.3546 - val_loss: 6.4276 - val_accuracy

In [26]:
# save model
model.save('next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [27]:
history

{'accuracy': [0.12561781704425812,
  0.17970100045204163,
  0.20835959911346436,
  0.23961150646209717,
  0.27618223428726196,
  0.3147869408130646,
  0.3545713424682617,
  0.39215904474258423,
  0.42722466588020325,
  0.45655447244644165,
  0.48480626940727234,
  0.5080239772796631,
  0.5290348529815674,
  0.5478084087371826,
  0.5622190833091736,
  0.5759483575820923,
  0.588009774684906,
  0.5985457301139832,
  0.6070578694343567,
  0.6145632266998291,
  0.623095691204071,
  0.6300925612449646,
  0.6361232399940491,
  0.6403539180755615,
  0.6461710333824158,
  0.6515305638313293,
  0.6557612419128418,
  0.6600732207298279,
  0.6627072095870972,
  0.6683412790298462,
  0.6697549223899841,
  0.6735787391662598,
  0.6754500269889832,
  0.6777890920639038,
  0.6803315281867981,
  0.682914674282074,
  0.6839418411254883,
  0.6868504285812378,
  0.688294529914856,
  0.6893725395202637,
  0.6914980411529541,
  0.6933285593986511,
  0.6940709948539734,
  0.6957286596298218,
  0.69768124818

## Load Trained Model

In [28]:
# load model
model = load_model('/content/next_word_model.h5')

## Predict next words

In [29]:
def prepare_input(text):
    words = tokenizer.tokenize(text)
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(words):
        x[0, t, word2id[word]] = 1.
        
    return x

In [30]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [31]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_word = id2word[next_index]
        text = text[1:] + next_word
        completion += next_word
        
        if len(original_text + completion) + 2 > len(original_text) and next_word == ' ':
            return completion

In [32]:
def predict_next_word(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [id2word[idx] for idx in next_indices]

### Testing

In [33]:
input="I look forward to working"
print(predict_next_word(input))

['with', 'this', 'here']


In [34]:
input="I look forward to working"
print(predict_next_word(input, 5))

['with', 'this', 'here', 'pretty', 'They']


In [35]:
input="working with you about "
print(predict_next_word(input, 2))

['But', 'next']


In [36]:
input="I would like to tell"
print(predict_next_word(input, 2))

['you', 'him']


In [40]:
input="To working with you about"
print(predict_next_word(input, 2))

['this', 'an']
